In [1]:
!pip install -q -U "transformers>=4.45.0,<5.0.0" huggingface_hub datasets peft trl bitsandbytes accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 77.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.5 which is incompatible.


In [2]:
import wandb
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
# Add these tokens in Kaggle's "Secrets" tab (Add-ons -> Secrets)
login(token=user_secrets.get_secret("HF_TOKEN"))
wandb.login(key=user_secrets.get_secret("WANDB_API_KEY"))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shreyashgaur01 (Nyaya-LLM) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
import torch
import os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

os.environ["WANDB_ENTITY"] = "Nyaya-LLM"
os.environ["WANDB_PROJECT"] = "nyaya-llm-phase1"

MODEL_NAME = "google/gemma-3-4b-it"
DATA_PATH = "/kaggle/input/datasets/shreyashgaur/nyaya-llm-dataset/all_acts_train.jsonl"
VAL_PATH = "/kaggle/input/datasets/shreyashgaur/nyaya-llm-dataset/all_acts_val.jsonl"
OUTPUT_DIR = "/kaggle/working/results/qlora_phase1_gemma3_4b"
MAX_SEQ_LENGTH = 1024


def format_prompt(example):
    return f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"


# Gemma-3 specific: prevents multimodal crash by injecting token_type_ids
class GemmaSFTTrainer(SFTTrainer):
    def training_step(self, model, inputs, num_items_in_batch=None):
        if "token_type_ids" not in inputs:
            inputs["token_type_ids"] = torch.zeros_like(inputs["input_ids"])
        return super().training_step(model, inputs, num_items_in_batch)

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        if "token_type_ids" not in inputs:
            inputs["token_type_ids"] = torch.zeros_like(inputs["input_ids"])
        return super().prediction_step(model, inputs, prediction_loss_only, ignore_keys)


def main():
    print("Loading dataset...")
    dataset = load_dataset("json", data_files={"train": DATA_PATH, "validation": VAL_PATH})

    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=False)
    tokenizer.pad_token = tokenizer.eos_token

    print("Configuring 4-bit Quantization (QLoRA)...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    print("Loading base model in 4-bit...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=False,
        dtype=torch.float16,
    )

    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)
    model.config.use_cache = False

    print("Applying LoRA adapters...")
    peft_config = LoraConfig(
        r=32,
        lora_alpha=64,
        target_modules="all-linear",     # matches phi QLoRA exactly
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

    print("Setting up Trainer...")
    training_args = SFTConfig(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,   # effective batch = 8, same as before
        eval_strategy="steps",
        eval_steps=100,
        logging_steps=25,
        num_train_epochs=2,
        learning_rate=2e-4,
        fp16=False,                      # ← the actual fix
        bf16=False,
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        save_total_limit=4,
        max_length=MAX_SEQ_LENGTH,
        report_to="wandb",
        run_name="qlora-<model>",
        dataloader_pin_memory=False,
    )

    trainer = GemmaSFTTrainer(
        model=model,
        processing_class=tokenizer,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        formatting_func=format_prompt,
        args=training_args,
    )

    print("Starting training...")
    trainer.train()

    print("Saving model adapter...")
    trainer.model.save_pretrained(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    print("Gemma-3-4B QLoRA Training Complete.")


if __name__ == "__main__":
    main()

2026-02-19 07:30:53.856045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771486254.031847      23 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771486254.082863      23 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771486254.513248      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771486254.513290      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771486254.513293      23 computation_placer.cc:177] computation placer alr

Loading dataset...


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Loading tokenizer...


...

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 1}.


Starting training...


wandb: WARNING Changes to your `wandb` environment variables will be ignored because your `wandb` session has already started. For more information on how to modify your settings with `wandb.init()` arguments, please refer to https://wandb.me/wandb-init.
wandb: Tracking run with wandb version 0.25.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20260219_073211-4zdppxoc
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run qlora-<model>
wandb: ⭐️ View project at https://wandb.ai/Nyaya-LLM/nyaya-llm-phase1
wandb: 🚀 View run at https://wandb.ai/Nyaya-LLM/nyaya-llm-phase1/runs/4zdppxoc


Step,Training Loss,Validation Loss
100,1.186400,1.143661
200,1.070200,1.055996
300,0.983300,0.984773
400,0.965700,0.904245
500,0.867100,0.830187
600,0.804500,0.752231
700,0.712300,0.696381
800,0.644800,0.639703
900,0.608300,0.578101
1000,0.392200,0.537110


Saving model adapter...
Gemma-3-4B QLoRA Training Complete.
